# 1. ライブラリのインポート

PytorchのMNISTのサンプルをコピーしただけなのでいらないものもあるかもしれません。

In [1]:
from __future__ import print_function
import argparse
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# 2. モデルの定義

サンプルのまんまです。

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


# 3. 学習データの準備

サンプルのまんまです。

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
        transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64)

100.1%

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


113.5%

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


# 4. seedの設定

サンプルのまんまです。

In [4]:
torch.manual_seed(1)

# 5. 学習を実行する関数

サンプルをもとに記載しています。


In [5]:
def train(mode):
    device = torch.device(mode)
    
    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=1.0)
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if mode == "cuda" else {}
    
    model.train()

    start = time.time()
    
    for epoch in range(1, 15):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    end = time.time()
    
    processing_time = end - start
    print("Pprocessing Time(sec): " + str(processing_time))

# 6. 学習を実行

## 6-1. GPUの場合

GPU利用の場合です。

In [6]:
train("cuda")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.283209
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.259838
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.938267
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.519191
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.457256
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.375608
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.295962
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.340620
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.378330
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.203926
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.249873
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.272896
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.317023
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.102728
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.172263
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.131872
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.430769
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.189571
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.528159
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.151123
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.177763
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.127040
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.083575
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.066318
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.028740
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.104123
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.074258
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.055830
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.156342
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.012080
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.007193
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.144706
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.038759
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.004630
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.004843
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.094848
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.024128
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.119238
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.083051
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.006607


Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.108302
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.007671
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.115628
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.014749
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.003612
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.191276
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.039495
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.048552
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.036249
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.129684
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.059538
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.003590
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.061175
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.031803
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.023667
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.031110
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.012630
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.034004
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.033044
Train Epoch: 4 [44160/60000 (74%)]	Loss: 0.011192


Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.012419
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.000766
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.069191
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.132058
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.003510
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.106907
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.024607
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.057770
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.007545
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.020263
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.010359
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.013564
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.024551
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.009271
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.061180
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.116354
Train Epoch: 6 [28160/60000 (47%)]	Loss: 0.005892
Train Epoch: 6 [28800/60000 (48%)]	Loss: 0.016745
Train Epoch: 6 [29440/60000 (49%)]	Loss: 0.043387
Train Epoch: 6 [30080/60000 (50%)]	Loss: 0.121171


Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.005557
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.017211
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.040885
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.051730
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.027293
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.135057
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.127506
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.006996
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.007442
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.021449
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.036229
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.101195
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.002899
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.007431
Train Epoch: 8 [12160/60000 (20%)]	Loss: 0.004340
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.025253
Train Epoch: 8 [13440/60000 (22%)]	Loss: 0.032501
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.001551
Train Epoch: 8 [14720/60000 (25%)]	Loss: 0.061483
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.000166
Train Epoch: 8 

Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.003592
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.000017
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.035573
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.016749
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.093947
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.005381
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.002989
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.069417
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.022657
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.107066
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.004622
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.046667
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.024424
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.044704
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.002613
Train Epoch: 9 [58240/60000 (97%)]	Loss: 0.000114
Train Epoch: 9 [58880/60000 (98%)]	Loss: 0.000164
Train Epoch: 9 [59520/60000 (99%)]	Loss: 0.000015
Train Epoch: 10 [0/60000 (0%)]	Loss: 0.009505
Train Epoch: 10 [640/60000 (1%)]	Loss: 0.000171
Train 

Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.006037
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.001109
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.000060
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.001618
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.034683
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.035553
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.067165
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.001114
Train Epoch: 11 [37760/60000 (63%)]	Loss: 0.068951
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.002926
Train Epoch: 11 [39040/60000 (65%)]	Loss: 0.004801
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.001623
Train Epoch: 11 [40320/60000 (67%)]	Loss: 0.005367
Train Epoch: 11 [40960/60000 (68%)]	Loss: 0.031884
Train Epoch: 11 [41600/60000 (69%)]	Loss: 0.001066
Train Epoch: 11 [42240/60000 (70%)]	Loss: 0.013724
Train Epoch: 11 [42880/60000 (71%)]	Loss: 0.019707
Train Epoch: 11 [43520/60000 (72%)]	Loss: 0.062044
Train Epoch: 11 [44160/60000 (74%)]	Loss: 0.000372
Train Epoch: 11 [44800/60000 (7

Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.003238
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.067206
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.001363
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.048032
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.001376
Train Epoch: 13 [19200/60000 (32%)]	Loss: 0.009487
Train Epoch: 13 [19840/60000 (33%)]	Loss: 0.001598
Train Epoch: 13 [20480/60000 (34%)]	Loss: 0.009862
Train Epoch: 13 [21120/60000 (35%)]	Loss: 0.005852
Train Epoch: 13 [21760/60000 (36%)]	Loss: 0.000003
Train Epoch: 13 [22400/60000 (37%)]	Loss: 0.027169
Train Epoch: 13 [23040/60000 (38%)]	Loss: 0.078476
Train Epoch: 13 [23680/60000 (39%)]	Loss: 0.028048
Train Epoch: 13 [24320/60000 (41%)]	Loss: 0.000184
Train Epoch: 13 [24960/60000 (42%)]	Loss: 0.000461
Train Epoch: 13 [25600/60000 (43%)]	Loss: 0.007881
Train Epoch: 13 [26240/60000 (44%)]	Loss: 0.000732
Train Epoch: 13 [26880/60000 (45%)]	Loss: 0.065877
Train Epoch: 13 [27520/60000 (46%)]	Loss: 0.000050
Train Epoch: 13 [28160/60000 (4

Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.000002


TypeError: must be str, not float

## 6-2. CPUの場合

CPUのみ利用の場合です。

In [ ]:
train("cpu")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.281064
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.362333
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.775346
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.496984
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.559840
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.407202
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.189583
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.354673
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.554514
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.407726
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.346386
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.356277
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.165873
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.298102
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.234378
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.157134
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.336546
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.099107
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.593733
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.195508
Train Epoch: 1 [12800/60000 (